In [ ]:
!pip install summac

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.2
    Uninstalling huggingface-hub-0.20.2:
      Successfully uninstalled huggingface-hub-0.20.2


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from summac.model_summac import SummaCZS, SummaCConv

model_zs = SummaCZS(granularity="sentence", model_name="vitc", device="cpu") # If you have a GPU: switch to: device="cuda"
model_conv = SummaCConv(models=["vitc"], bins='percentile', granularity="sentence", nli_labels="e", device="cpu", start_file="default", agg="mean")

document = """SUBJECT: The results of smoking
MESSAGE: What are some illnesses caused by smoking? Also, what are their effect on the human body?"""

summary1 = "What are some illnesses caused by smoking? Also, what are their effect on the human body?"
score_zs1 = model_zs.score([document], [summary1])
score_conv1 = model_conv.score([document], [summary1])
print("[Summary 1] SummaCZS Score: %.3f; SummacConv score: %.3f" % (score_zs1["scores"][0], score_conv1["scores"][0])) # [Summary 1] SummaCZS Score: 0.582; SummacConv score: 0.536

# summary2 = "There are strange shape patterns on Arcadia Planitia. The shapes could indicate the area might be made of glaciers."
# score_zs2 = model_zs.score([document], [summary2])
# score_conv2 = model_conv.score([document], [summary2])
# print("[Summary 2] SummaCZS Score: %.3f; SummacConv score: %.3f" % (score_zs2["scores"][0], score_conv2["scores"][0])) # [Summary 2] SummaCZS Score: 0.877; SummacConv score: 0.709

<All keys matched successfully>
[Summary 1] SummaCZS Score: 0.965; SummacConv score: 0.673


In [ ]:
from tqdm import tqdm
import pandas as pd
from summac.model_summac import SummaCZS, SummaCConv

# Load the CSV file
df = pd.read_csv("Bangla1.csv")

# Extract the predictions and references columns
predictions = df["New"].tolist()
references = df["question"].tolist()

# Initialize the SummaC evaluators
model_zs = SummaCZS(granularity="sentence", model_name="vitc", device="cpu")  # If you have a GPU: switch to: device="cuda"
model_conv = SummaCConv(models=["vitc"], bins='percentile', granularity="sentence", nli_labels="e", device="cpu", start_file="default", agg="mean")

# Compute the SummaC scores for each prediction-reference pair
scores_zs = []
scores_conv = []
n_predictions = len(predictions)
with tqdm(total=n_predictions) as pbar:
    for i in range(n_predictions):
        # Compute the SummaC scores for the current prediction-reference pair
        score_zs = model_zs.score([predictions[i]], [references[i]])
        score_conv = model_conv.score([predictions[i]], [references[i]])

        # Print the immediate scores
        print(f"SummaCZS Score for prediction {i+1}: {score_zs['scores'][0]:.3f}")
        print(f"SummaCConv Score for prediction {i+1}: {score_conv['scores'][0]:.3f}")

        # Append the scores to the list
        scores_zs.append(score_zs["scores"][0])
        scores_conv.append(score_conv["scores"][0])

        # Update the progress bar
        pbar.update(1)
        pbar.set_description("Processing predictions... (%d/%d)" % (i+1, n_predictions))

# Compute the average scores
avg_score_zs = sum(scores_zs) / len(scores_zs)
avg_score_conv = sum(scores_conv) / len(scores_conv)

# Print the average scores
print("Average SummaCZS Score: %.3f; Average SummacConv score: %.3f" % (avg_score_zs, avg_score_conv))


<All keys matched successfully>


  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/summac/model_summac.py:298: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  histograms = torch.FloatTensor(histograms).to(self.device)
Processing predictions... (1/3):  33%|███▎      | 1/3 [00:24<00:48, 24.16s/it]

SummaCZS Score for prediction 1: 0.347
SummaCConv Score for prediction 1: 0.222


Processing predictions... (2/3):  67%|██████▋   | 2/3 [00:48<00:23, 23.99s/it]

SummaCZS Score for prediction 2: 0.507
SummaCConv Score for prediction 2: 0.218


Processing predictions... (3/3): 100%|██████████| 3/3 [01:11<00:00, 23.86s/it]

SummaCZS Score for prediction 3: 0.230
SummaCConv Score for prediction 3: 0.214
Average SummaCZS Score: 0.361; Average SummacConv score: 0.218
